In [1]:
# clone repository for deepsort with yolov4
# forked from theAIGuysCode/yolov4-deepsort
# !git clone https://github.com/zyerusha/yolov4-deepsort  

In [2]:
# uncomment below line ONLY if needed
#!pip install -r requirements-gpu.txt

In [3]:
from shutil import copy2
src_dir = '../yolo/weights/yolov4.weights'
tgt_dir = './yolov4.weights'
copy2(src_dir, tgt_dir)

'./yolov4.weights'

In [4]:
# Convert darknet weights to tensorflow model
%cd tensorflow_yolov4_tflite/
# !python save_model.py --model yolov4 --weights ../yolov4.weights --output ../checkpoints/yolov4
%cd ..

c:\Users\Zeph\Documents\AI_ML\video_velocity_finder\tensorflow_yolov4_tflite
c:\Users\Zeph\Documents\AI_ML\video_velocity_finder


In [5]:
import shutil

def CopyNeededFolders(folder_name, src_dir, target_dir = '.'):
    tgt_name = os.path.join(target_dir, folder_name)
    src_name = os.path.join(src_dir, folder_name)
    if os.path.isdir(tgt_name): 
        shutil.rmtree(tgt_name) 
    shutil.copytree(src_name,tgt_name)
    print(f'Copied folder {src_name} --> {tgt_name}')

CopyNeededFolders('data', './tensorflow_yolov4_tflite', '.') 
CopyNeededFolders('core', './tensorflow_yolov4_tflite', '.') 
CopyNeededFolders('model_data', '../yolov4-deepsort', '.') 


Copied folder ./tensorflow_yolov4_tflite\data --> .\data
Copied folder ./tensorflow_yolov4_tflite\core --> .\core
Copied folder ../yolov4-deepsort\model_data --> .\model_data


In [6]:
# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt_'
yolo_video_name = 'yolo_'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


src_video = os.path.join(video_src_path, video_name_orig)
gt_video_name = yml_video_name + video_name + '.mp4'#video_ext
yolo_video_name = yolo_video_name + video_name + '.mp4'#video_ext

In [7]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df

In [8]:
import pandas as pd

# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

Loading annotations...


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category
0,0.0,0.0,0.000000,485,743,653,914,Vehicle
1,0.0,1.0,0.033333,489,748,657,919,Vehicle
2,0.0,2.0,0.066667,488,747,656,918,Vehicle
3,0.0,3.0,0.100000,488,747,656,918,Vehicle
4,0.0,4.0,0.133333,488,747,656,918,Vehicle


In [9]:
import cv2
from utils.velocity_utils import VelocityUtils
velUtils = VelocityUtils() 

vidcap = cv2.VideoCapture(src_video)
fps = 30
scale = 1/15
if vidcap.isOpened():
    fps = vidcap.get(cv2.CAP_PROP_FPS)  
    print(fps)

df_bbox = velUtils.add_vel_to_df(df_bbox, fps, scale)

df_bbox.head()

29.97002997002997


,object_id,frame_id,time_sec,bb_left,bb_top,bb_right,bb_bottom,category,x,y,dx,dy,vx,vy,vel
0,0.0,0.0,0.000000,485,743,653,914,Vehicle,828.5,569.0,5.0,4.0,9.990010,7.992008,12.79
1,0.0,1.0,0.033333,489,748,657,919,Vehicle,833.5,573.0,5.0,4.0,9.990010,7.992008,12.79
2,0.0,2.0,0.066667,488,747,656,918,Vehicle,832.5,572.0,-1.0,-1.0,-1.998002,-1.998002,2.83
3,0.0,3.0,0.100000,488,747,656,918,Vehicle,832.5,572.0,0.0,0.0,0.000000,0.000000,0.00
4,0.0,4.0,0.133333,488,747,656,918,Vehicle,832.5,572.0,0.0,0.0,0.000000,0.000000,0.00


In [10]:
import os
from utils.yolo_utils import YoloUtils
from utils.video_utils import VideoUtils
from utils.folder_utils import FolderUtils
from utils.deepsort_yolo import DeepsortYolo
yUtils = YoloUtils()
vUtils = VideoUtils() 
deepsortYolo  = DeepsortYolo()

yolo_weight_file = "../yolo/weights/yolov3.weights"
yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../yolo/darknet/data/coco.names"
# yolo_weight_file = "yolov4.weights"
# yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../yolo/darknet/data/coco.names"
start_time = 0
video_duration=60#None
video_in = cv2.VideoCapture(src_video)
if video_in.isOpened():
    fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
    start_count, end_count = vUtils.GetStartEndCount(fps, total_frames, start_time, video_duration)
    video_duration = int(end_count / fps)


yolo_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(yolo_video_name, start_time, video_duration))
if os.path.exists(yolo_filename):
    os.remove(yolo_filename)

gt_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(gt_video_name, start_time, video_duration))
if os.path.exists(gt_filename):
    os.remove(gt_filename)

df_bbox_vehicle = df_bbox[df_bbox['category'] == 'Vehicle']
yolo_video_out, bbox_yolo = deepsortYolo.ProcessVideo("./checkpoints/yolov4", yolo_names_file, src_video, video_dest_path, yolo_filename, start_time_sec=start_time, duration_sec=video_duration, save_images=False)
gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, yolo_video_out, gt_filename, df_bbox_vehicle, start_time, video_duration)


Total frames in video: 3351 @ 30 frames/sec
../datasets/VIRAT/Videos/Ground/VIRAT_S_050000_07_001014_001126.mp4
opened video
Total frames in video: 3351 @ 30 frames/sec
Created frame id 25, 0.83 sec in video; Objects Cnt: 8 completed:  1.4 %
Created frame id 50, 1.67 sec in video; Objects Cnt: 8 completed:  2.8 %
Created frame id 75, 2.50 sec in video; Objects Cnt: 10 completed:  4.2 %
Created frame id 100, 3.33 sec in video; Objects Cnt: 10 completed:  5.6 %
Created frame id 125, 4.17 sec in video; Objects Cnt: 9 completed:  6.9 %
Created frame id 150, 5.00 sec in video; Objects Cnt: 10 completed:  8.3 %
Created frame id 175, 5.83 sec in video; Objects Cnt: 9 completed:  9.7 %
Created frame id 200, 6.67 sec in video; Objects Cnt: 9 completed:  11.1 %
Created frame id 225, 7.50 sec in video; Objects Cnt: 9 completed:  12.5 %
Created frame id 250, 8.33 sec in video; Objects Cnt: 10 completed:  13.9 %
Created frame id 275, 9.17 sec in video; Objects Cnt: 9 completed:  15.3 %
Created fram

In [11]:
# print(bbox_gt)
# print(bbox_yolo)

In [12]:
from utils.bbox_utils import Bbox
import pandas as pd

def GetMaxCorrelation(bb_gt, pred_bboxes, frame):
    max_iou = 0.0
    max_info = [frame, bb_gt, bb_gt, max_iou]
    found = False

    for bb_pred in pred_bboxes:
        gt = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        pred = Bbox(bb_pred[0],bb_pred[1],bb_pred[2],bb_pred[3])
        iou = gt.IOU(pred)

        if (max_iou < iou) & (0.5 < iou):
            found = True
            max_iou = iou
            max_info = [frame, bb_gt, bb_pred, iou]

    return found, max_info

df = pd.DataFrame(columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
print(f'# of frames in video: {len(bbox_gt)}')
# for i_frame in range(1):#range(len(bbox_gt)): #scan all frames in video
for i_frame in range(len(bbox_gt)): #scan all frames in video
    data = []
    for bb_gt in bbox_gt[i_frame]:
        found, info = GetMaxCorrelation(bb_gt, bbox_yolo[i_frame], i_frame)
        if(found):
            data.append(info)
    
    df_frame = pd.DataFrame(data, columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
    df = pd.concat([df, df_frame], ignore_index=True)



mPA = df['iou'].sum()/len(df)
print(mPA)
df.head(30)


# of frames in video: 1801
0.7781625018851617


,frame idx,gt bbox,pred bbox,iou
0,2,"[737, 41, 873, 140]","[748.9250923236009, 42.983385464654454, 862.09...",0.810053
1,2,"[792, 117, 914, 202]","[796.4368374483802, 120.99999999999999, 908.56...",0.810424
2,2,"[872, 203, 994, 298]","[881.3921345158753, 203.22044464687298, 991.37...",0.787412
3,2,"[917, 277, 1032, 365]","[921.0, 276.0, 1027.0, 365.0]",0.912952
4,2,"[981, 352, 1128, 469]","[978.9308030143414, 346.9834354828531, 1124.08...",0.886814
5,2,"[1055, 441, 1187, 557]","[1070.4041559134516, 457.0, 1179.375399439675,...",0.636040
6,2,"[1108, 510, 1240, 626]","[1116.2034705828244, 524.7765653389725, 1235.5...",0.726529
7,2,"[1190, 604, 1322, 720]","[1196.688906354448, 623.983150400066, 1323.311...",0.773264
8,3,"[1387, 1015, 1645, 1079]","[1375.3080949400712, 1022.4274182625417, 1662....",0.793872
9,3,"[737, 41, 873, 140]","[749.3434524611578, 43.71392658867786, 861.704...",0.798387


In [13]:
# add IOU to video
from video_utils_virat import VideoUtils
vUtils = VideoUtils() 

gt_video_out

def AddIouToFrame(img, df_info):
    for index, row in df_info.iterrows():
        bb_gt = row['gt bbox']
        txt_val = "{iou:.4f}".format(iou = row['iou'])
        bb = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        img = vUtils.PrintText(img, txt_val, bb.center_x, bb.center_y, -10, 10, cv2.FONT_HERSHEY_COMPLEX, 0.75, (255, 255, 255))
    return img


def AddIouToVideo(output_dir, orig_v_full_path, df_info):
    if (not orig_v_full_path):
        raise Exception(f"File not found: {orig_v_full_path}")

    bbox_data = {}
    # Open original video
    video_in = cv2.VideoCapture(orig_v_full_path)
    if video_in.isOpened():
        fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
        count = 0
        # setting CV_CAP_PROP_POS_FRAMES at count
        video_in.set(cv2.CAP_PROP_POS_FRAMES, count)
        full_filename = os.path.join(output_dir,'IOU.mp4')
        if os.path.exists(full_filename):
            os.remove(full_filename)

        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        video_out = cv2.VideoWriter(
            full_filename, fourcc, int(fps), frame_size)

        i = 0
        while (True):
            success, img = video_in.read()
            if(success):
                height, width, layers = img.shape
                image_size = (width, height)

                # All bounding box info for this frame
                # Add all annotations to the frame
                df_img = df_info[df_info['frame idx'] == count]
                img = AddIouToFrame(img, df_img)
                i += 1
                video_out.write(img)
                count += 1

            else:
                break

        video_in.release()  # done with original video
        video_out.release()

        print("Done: Created video: " + full_filename)

    cv2.destroyAllWindows()


AddIouToVideo(video_dest_path, gt_video_out, df)


Total frames in video: 1801.00 @ 30.00 frames/sec
Done: Created video: ./processed/VIRAT_S_050000_07_001014_001126/IOU.mp4


In [14]:
if os.path.isfile("./test.avi"):
    os.remove("./test.avi")

# !python car_tracker.py --video ../datasets/VIRAT/Videos/Ground/VIRAT_S_050000_09_001310_001373.mp4 --output ./test.avi --model yolov4 --dont_show

In [15]:
# video_file = "VIRAT_S_050000_09_001310_001373.mp4"
# src_path_video = ../../datasets/VIRAT/Videos/Ground/VIRAT_S_050000_09_001310_001373.mp4 # os.path.join("../../datasets/VIRAT/Videos/Ground/",video_file)
# tgt_path_video = ./outputs/VIRAT_S_050000_09_001310_001373.avi  #os.path.join("./outputs/",video_file)
# %cd yolov4-deepsort/
# !python object_tracker.py --video src_path_video --output tgt_path_video --model yolov4 --dont_show

In [16]:
from video_utils_virat import YoloUtils
yUtils = YoloUtils()
yolo_weight_file = "../yolo/weights/yolov3.weights"
yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../yolo/darknet/data/coco.names"
yUtils.YoloOnScreen(yolo_weight_file, yolo_cfg_file, yolo_names_file, 0,0,800,600)


error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\layers\convolution_layer.cpp:387: error: (-2:Unspecified error) Number of input channels should be multiple of 3 but got 4 in function 'cv::dnn::ConvolutionLayerImpl::getMemoryShapes'
